In [ ]:
import torch.nn as nn
class BasicBlock(nn.Module):
    multiplier=1
    def __init__(self, input_num_planes, num_planes, strd=1):
        super().__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=input_num_planes,
                                    out_channels=num_planes,
                                     kernel_size=3,
                                     stride=stride,
                                     padding=1, bias=False) #padding =1 'same'
        self.batch_norm1 = nn.BatchNorm2d(num_planes)
        self.conv_layer2 = nn.Conv2d(in_channels=num_planes,
                                     out_channels=num_planes,
                                     kernel_size=3,
                                     stride=1, padding=1, bias=False) # 채널수변동 없음
        self.batch_norm2 = nn.BatchNorm2d(num_planes)
        # short cut 구현
        self.res_connnection = nn.Sequential() # 초기화

        # 입출력의 크기가 다른 경우 (채널과 featuremap size)맞춰주는 과정
        # stride 의 크기가 1보다 큰경우 --> feature map 의 사이즈가 변함.
        # 따라서 ip 의 사이즈를 ot 과 맞춰주는 과정이 필요하다.
        if strd > 1 or input_num_planes != self.multiplier*num_planes:
            self.res_connnection = nn.Sequential(
                nn.Conv2d(in_channels=input_num_planes,
                          out_channels=self.multiplier*num_planes,
                          # kernel_size=1, stride=strd 를 이용하여 맞춰준다.
                          kernel_size=1,
                          stride=strd, bias=False),
                nn.BatchNorm2d(self.multiplier*num_planes)
            )
    def forward(self, inp):
        op = F.relu(self.batch_norm1(self.conv_layer1(inp)))
        op = self.batch_norm2(self.conv_layer2(op))
        op += self.res_connnection(inp) # 사이즈를 변경한 ip 과 op 더함
        op = F.relu(op) # relu
        return op